<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [1]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [2]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.20.1
Pandas:  1.2.4


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [3]:
# wrong:
bikes = pd.read_csv('bikeshare.csv', header = None)
print(bikes)

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 0)
print(bikes.head())

                        0       1        2           3        4      5   \
0                 datetime  season  holiday  workingday  weather   temp   
1      2011-01-01 00:00:00       1        0           0        1   9.84   
2      2011-01-01 01:00:00       1        0           0        1   9.02   
3      2011-01-01 02:00:00       1        0           0        1   9.02   
4      2011-01-01 03:00:00       1        0           0        1   9.84   
...                    ...     ...      ...         ...      ...    ...   
10882  2012-12-19 19:00:00       4        0           1        1  15.58   
10883  2012-12-19 20:00:00       4        0           1        1  14.76   
10884  2012-12-19 21:00:00       4        0           1        1  13.94   
10885  2012-12-19 22:00:00       4        0           1        1  13.94   
10886  2012-12-19 23:00:00       4        0           1        1  13.12   

           6         7          8       9           10     11  
0       atemp  humidity  windspeed 

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [4]:
#ANSWER:
bikes = pd.read_csv('bikeshare.csv', header = 'infer')
print(bikes.head())
print()

              datetime  season  holiday  workingday  weather  temp   atemp  \
0  2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1  2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2  2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3  2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4  2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  
0        81        0.0       3          13     16  
1        80        0.0       8          32     40  
2        80        0.0       5          27     32  
3        75        0.0       3          10     13  
4        75        0.0       0           1      1  



In [5]:
type(bikes.shape)
(bikes.columns)

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count'],
      dtype='object')

Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [6]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [7]:
df = pd.read_excel('Iris.xls', sheet_name = 'Iris', header =(1)) 
df.head()

,Unnamed: 0,Sepal Length (cm),Sepal Width (cm),Petal Length (cm),Petal Width (cm),Class,Unnamed: 6,Unnamed: 7,alpha,obj,Unnamed: 10,Unnamed: 11
0,NaN,7.0,3.2,4.7,1.4,Iris-versicolor,NaN,0.0,0.0,0.0,0,1
1,NaN,6.4,3.2,4.5,1.5,Iris-versicolor,NaN,0.0,NaN,NaN,0,1
2,NaN,6.9,3.1,4.9,1.5,Iris-versicolor,NaN,0.0,NaN,NaN,0,1
3,NaN,5.5,2.3,4.0,1.3,Iris-versicolor,NaN,0.0,NaN,NaN,0,1
4,NaN,6.5,2.8,4.6,1.5,Iris-versicolor,NaN,NaN,NaN,NaN,0,1


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [8]:
#ANSWER
df = pd.read_excel('Iris.xls', sheet_name = 'Iris', header =(1),usecols='B:F,I,J') 
print (df)



    Sepal Length (cm)  Sepal Width (cm)  Petal Length (cm)  Petal Width (cm)  \
0                 7.0               3.2                4.7               1.4   
1                 6.4               3.2                4.5               1.5   
2                 6.9               3.1                4.9               1.5   
3                 5.5               2.3                4.0               1.3   
4                 6.5               2.8                4.6               1.5   
..                ...               ...                ...               ...   
95                4.8               3.0                1.4               0.3   
96                5.1               3.8                1.6               0.2   
97                4.6               3.2                1.4               0.2   
98                5.3               3.7                1.5               0.2   
99                5.0               3.3                1.4               0.2   

              Class  alpha  obj  
0   I

### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [9]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [10]:
url = 'https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/'

df = pd.read_html(url)



#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [11]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'
# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [12]:
#ANSWER

x=bikes.shape


What are the column names in `bikes`?

In [13]:
#ANSWER
bikes.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count'],
      dtype='object')

What are the data types of these columns?

In [14]:
bikes.dtypes

datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windspeed     float64
casual          int64
registered      int64
count           int64
dtype: object

What is the (row) index for this DataFrame?

In [15]:
#ANSWER
bikes.index

RangeIndex(start=0, stop=10886, step=1)

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [26]:
#ANSWER
x=bikes['holiday']
print(type(x))

<class 'pandas.core.series.Series'>


In [27]:
y=bikes[['holiday']]
print(type(y))

<class 'pandas.core.frame.DataFrame'>


<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>

How would we use object notation to show the first 4 rows of `atemp`?

In [30]:
#ANSWER
bikes.iloc[0:4 ,6]

0    14.395
1    13.635
2    13.635
3    14.395
Name: atemp, dtype: float64

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [42]:
bikes[[0]]
#E: bikes[0]
#E: bikes[0,0]
#E: bikes[[0,0]]

,0
0,datetime
1,2011-01-01 00:00:00
2,2011-01-01 01:00:00
3,2011-01-01 02:00:00
4,2011-01-01 03:00:00
...,...
10882,2012-12-19 19:00:00
10883,2012-12-19 20:00:00
10884,2012-12-19 21:00:00
10885,2012-12-19 22:00:00


What is the correct way to access the 1st row of the DataFrame by its index?

In [35]:
#ANSWER

#row1 =df.iloc[1]
#print(row1)

row1 = bikes.iloc[0, :]
print(row1)


datetime      2011-01-01 00:00:00
season                          1
holiday                         0
workingday                      0
weather                         1
temp                         9.84
atemp                      14.395
humidity                       81
windspeed                     0.0
casual                          3
registered                     13
count                          16
Name: 0, dtype: object


What is the correct way to access the 2nd column of the DataFrame by its index?

In [36]:
#ANSWER
col2 = bikes.iloc[0: , 1]
print(col2)

0        1
1        1
2        1
3        1
4        1
        ..
10881    4
10882    4
10883    4
10884    4
10885    4
Name: season, Length: 10886, dtype: int64


## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER: The `isnull` function checks if the argument passed in is a Nan or a None or any other na values. Return True if the argument is Null

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [40]:
bikes.isnull().head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [43]:
#ANSWER
null_sum = bikes.isnull().sum()

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [49]:
#ANSWER
non_null_cols = [col for col,val in null_sum.iteritems() if val>0]

In [50]:
non_null_cols


[]

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as missing.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [69]:
#ANSWER
def scalar_div(x,y):
    if y==0:
        return 'y is 0. Division not possible'
    z=x/y
    return z

In [70]:
ans = scalar_div(1, 0)
ans

'y is 0. Division not possible'

Apply the Pandas `isna` function to the following data objects:

In [72]:
x = 2.3
y = np.nan
pd.isna(x)
pd.isna(y)

True

In [73]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
pd.isna(array)

array([[False,  True, False],
       [False, False,  True]])

How is the pandas I/O parameter `na_values` used?

? ANSWER: 
na_values is used to create a string that considers pandas as NaN (Not a Number).

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [75]:
#ANSWER
bikes = pd.read_csv('bikeshare.csv')
bikes.season.nunique()


4

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [81]:
#ANSWER
for col in ['instant', 'dteday', 'windspeed']:
    if col in bikes.columns:
        min_col = bikes[col].min()
        max_col = bikes[col].max()
        print('Column:', col, 'minimum:', min_col, 'maximum:', max_col)

Column: windspeed minimum: 0.0 maximum: 56.9969


Compute and print the overall minimum and maximum of the numeric data columns:

In [86]:
#bikes_min, bikes_max = (min(bikes.min()), max(bikes.max()))
bikes.describe()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
count,10886.000000,10886.000000,10886.000000,10886.000000,10886.00000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000
mean,2.506614,0.028569,0.680875,1.418427,20.23086,23.655084,61.886460,12.799395,36.021955,155.552177,191.574132
std,1.116174,0.166599,0.466159,0.633839,7.79159,8.474601,19.245033,8.164537,49.960477,151.039033,181.144454
min,1.000000,0.000000,0.000000,1.000000,0.82000,0.760000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,0.000000,1.000000,13.94000,16.665000,47.000000,7.001500,4.000000,36.000000,42.000000
50%,3.000000,0.000000,1.000000,1.000000,20.50000,24.240000,62.000000,12.998000,17.000000,118.000000,145.000000
75%,4.000000,0.000000,1.000000,2.000000,26.24000,31.060000,77.000000,16.997900,49.000000,222.000000,284.000000
max,4.000000,1.000000,1.000000,4.000000,41.00000,45.455000,100.000000,56.996900,367.000000,886.000000,977.000000


### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [91]:
bikes['atemp'].quantile(0.5)

24.24

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [88]:
#ANSWER
bikes.quantile([0.1, 0.25, 0.5, 0.75, 0.9])

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0.10,1.0,0.0,0.0,1.0,9.84,12.120,37.0,0.0000,1.0,7.0,9.0
0.25,2.0,0.0,0.0,1.0,13.94,16.665,47.0,7.0015,4.0,36.0,42.0
0.50,3.0,0.0,1.0,1.0,20.50,24.240,62.0,12.9980,17.0,118.0,145.0
0.75,4.0,0.0,1.0,2.0,26.24,31.060,77.0,16.9979,49.0,222.0,284.0
0.90,4.0,0.0,1.0,2.0,30.34,34.090,88.0,23.9994,94.0,354.0,452.0


### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [76]:
type(bikes['atemp'])

pandas.core.series.Series

In [77]:
bikes.sample(5)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
6234,2012-02-16 00:00:00,1,0,1,2,12.30,15.150,70,8.9981,1,22,23
10416,2012-11-19 10:00:00,4,0,1,2,14.76,16.665,66,16.9979,33,106,139
3802,2011-09-09 10:00:00,3,0,1,2,25.42,25.760,100,0.0000,2,27,29
679,2011-02-11 15:00:00,1,0,1,1,13.12,15.910,21,11.0014,12,62,74
5996,2012-02-06 01:00:00,1,0,1,1,9.02,12.880,75,6.0032,0,6,6


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [93]:
atemp_level = pd.cut(bikes['atemp'], bins = 5)
print(atemp_level)

0         (9.699, 18.638]
1         (9.699, 18.638]
2         (9.699, 18.638]
3         (9.699, 18.638]
4         (9.699, 18.638]
               ...       
10881    (18.638, 27.577]
10882     (9.699, 18.638]
10883     (9.699, 18.638]
10884     (9.699, 18.638]
10885     (9.699, 18.638]
Name: atemp, Length: 10886, dtype: category
Categories (5, interval[float64]): [(0.715, 9.699] < (9.699, 18.638] < (18.638, 27.577] < (27.577, 36.516] < (36.516, 45.455]]


What is `atemp_level`?

#ANSWER
atemp_level splits atemp(continous) into five different discrete bins

Here is a random sample of `atemp_level`:

In [94]:
atemp_level.sample(5)          

293      (9.699, 18.638]
7881    (27.577, 36.516]
4336    (18.638, 27.577]
726     (18.638, 27.577]
2064    (18.638, 27.577]
Name: atemp, dtype: category
Categories (5, interval[float64]): [(0.715, 9.699] < (9.699, 18.638] < (18.638, 27.577] < (27.577, 36.516] < (36.516, 45.455]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [95]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

2689    warm
8589    warm
6277    mild
6148    cool
475     cool
Name: atemp, dtype: category
Categories (4, object): ['cool' < 'mild' < 'warm' < 'hot']

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [96]:
#ANSWER
atemp_level = pd.cut(bikes['atemp'], bins = 2, labels = ["cool", "mild"]) # "warm", "hot", "Very Hot"])
atemp_level.sample(5)   

5309    cool
1260    cool
7532    mild
2596    mild
9492    mild
Name: atemp, dtype: category
Categories (2, object): ['cool' < 'mild']

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [94]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [119]:
print (a.shape, a)
print (b.shape, b)
print (c.shape, c)

(3,) [0. 0. 0.]
(1, 3) [[1. 1. 1.]]
(2, 3, 4) [[[6 2 7 9]
  [7 4 2 4]
  [1 1 3 7]]

 [[7 8 3 5]
  [7 5 4 5]
  [6 9 2 2]]]


In [123]:
e = np.array( [[1,2,3,4], [5,6,7,8]] )
print(e)

[[1 2 3 4]
 [5 6 7 8]]


In [124]:
e = np.array( [[1,2,3,4], [5,6,7,8]] )
print(e.shape, e)

(2, 4) [[1 2 3 4]
 [5 6 7 8]]


## Load Data

Load rock.csv and clean the dataset.

In [98]:
rock = pd.read_csv('rock.csv')
print(rock.head())
#rock['Release Year'].fillna(0,inplace=True)

#Release Year.fillna(value=0, inplace=True)
#{1:0}, inplace=True

               Song Clean ARTIST CLEAN Release Year  \
0        Caught Up in You  .38 Special         1982   
1            Fantasy Girl  .38 Special          NaN   
2         Hold On Loosely  .38 Special         1981   
3  Rockin' Into the Night  .38 Special         1980   
4       Art For Arts Sake         10cc         1975   

                                COMBINED  First?  Year?  PlayCount  F*G  
0        Caught Up in You by .38 Special       1      1         82   82  
1            Fantasy Girl by .38 Special       1      0          3    0  
2         Hold On Loosely by .38 Special       1      1         85   85  
3  Rockin' Into the Night by .38 Special       1      1         18   18  
4              Art For Arts Sake by 10cc       1      1          1    1  


## Check Column Names

Check column names and clean.

In [72]:
for col in rock.columns:
    print(col)

Song Clean
ARTIST CLEAN
Release Year
COMBINED
First?
Year?
PlayCount
F*G


## Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [100]:
#rock = rock.replace("Release Year" == "NaN", value=0)
#rock = rock.replace(np.nan,0)
rock['Release Year'] = rock['Release Year'].replace(np.nan,0)
print(rock)

                                       Song Clean ARTIST CLEAN Release Year  \
0                                Caught Up in You  .38 Special         1982   
1                                    Fantasy Girl  .38 Special            0   
2                                 Hold On Loosely  .38 Special         1981   
3                          Rockin' Into the Night  .38 Special         1980   
4                               Art For Arts Sake         10cc         1975   
...                                           ...          ...          ...   
2224                      She Loves My Automobile       ZZ Top            0   
2225                            Tube Snake Boogie       ZZ Top         1981   
2226                                         Tush       ZZ Top         1975   
2227                                   TV Dinners       ZZ Top         1983   
2228  WAITIN' FOR THE BUS/JESUS JUST LEFT CHICAGO       ZZ Top         1973   

                                               COMB

## Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [45]:
print(rock.dtypes)

Song Clean      object
ARTIST CLEAN    object
Release Year    object
COMBINED        object
First?           int64
Year?            int64
PlayCount        int64
F*G              int64
dtype: object


In [103]:
rock['Release Year'] = rock['Release Year'].replace('SONGFACTS.COM',0)
rock['Release Year']= rock['Release Year'].astype('int64')
print(rock.dtypes)                     

Song Clean      object
ARTIST CLEAN    object
Release Year     int64
COMBINED        object
First?           int64
Year?            int64
PlayCount        int64
F*G              int64
dtype: object


## Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [108]:
rock.describe().loc[['min','max']]

,Release Year,First?,Year?,PlayCount,F*G
min,0.0,1.0,0.0,0.0,0.0
max,2014.0,1.0,1.0,142.0,142.0


Release Year had an illogical Value 'SONGFACTS.COM', which has been replaced with a 0

## Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

## Apply these functions to your dataset

## Describe the new float-only DataFrame.

>




---



---



> > > > > > > > > © 2021 Institute of Data


---



---



